In [98]:
import urllib
import pyspark
import math
from pyspark.mllib.regression import LabeledPoint
from numpy import array
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from time import time
from spark_tree_plotting import plot_tree 
import json
import bson
from pyspark.sql import SQLContext
#from bson import json_util
#from bson.json_util import dumps

In [152]:
sc = pyspark.SparkContext('local[*]')


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-12-d9abf4af8e6c>:1 

In [ ]:
# getting train data
f = urllib.request.urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data.gz", "kddcup.data.gz")


data_file = "./kddcup.data.gz"
raw_data = sc.textFile(data_file)

print ("Train data size is {}".format(raw_data.count()))

In [100]:
# getting test data
ft = urllib.request.urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/corrected.gz", "corrected.gz")


test_data_file = "./corrected.gz"
test_raw_data = sc.textFile(test_data_file)

print("Test data size is {}".format(test_raw_data.count()))

Test data size is 311029


## Preparing the Data

In [101]:
raw_data.take(5)

['0,tcp,http,SF,215,45076,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,162,4528,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1,1,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,236,1228,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,2,2,1.00,0.00,0.50,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,233,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.00,0.00,0.00,0.00,1.00,0.00,0.00,3,3,1.00,0.00,0.33,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.00,0.00,0.00,0.00,1.00,0.00,0.00,4,4,1.00,0.00,0.25,0.00,0.00,0.00,0.00,0.00,normal.']

In [85]:
csv_data = raw_data.map(lambda x: x.split(","))
test_csv_data = test_raw_data.map(lambda x: x.split(","))

# get unique value of categorical variables
protocols = csv_data.map(lambda x: x[1]).distinct().collect() 
services = csv_data.map(lambda x: x[2]).distinct().collect()
flags = csv_data.map(lambda x: x[3]).distinct().collect()

In [105]:
csv_data.take(5)

[['0',
  'tcp',
  'http',
  'SF',
  '215',
  '45076',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '1',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '0',
  '0',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  'normal.'],
 ['0',
  'tcp',
  'http',
  'SF',
  '162',
  '4528',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '2',
  '2',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '1',
  '1',
  '1.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  'normal.'],
 ['0',
  'tcp',
  'http',
  'SF',
  '236',
  '1228',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '1',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '2',
  '2',
  '1.00',
  '0.00',
  

In [109]:
# create label point object for train data
def create_labeled_point(line_split):# convert string variable to categorical, its value = its index in unique list 
    # leave_out = [41]
    clean_line_split = line_split[0:41]
    
    # convert protocol to numeric categorical variable
    try: 
        clean_line_split[1] = protocols.index(clean_line_split[1])
    except:
        clean_line_split[1] = len(protocols)
        
    # convert service to numeric categorical variable
    try:
        clean_line_split[2] = services.index(clean_line_split[2])
    except:
        clean_line_split[2] = len(services)
    
    # convert flag to numeric categorical variable
    try:
        clean_line_split[3] = flags.index(clean_line_split[3])
    except:
        clean_line_split[3] = len(flags)
    
    # convert label to binary label
    attack = 1.0
    if line_split[41]=='normal.':
        attack = 0.0
    #data=spark.sparkContext.parallelize( np.concatenate((attack, clean_line_split.T), axis=1)) 
    return LabeledPoint(attack, array([float(x) for x in clean_line_split]))

training_data = csv_data.map(create_labeled_point)
test_data = test_csv_data.map(create_labeled_point)

In [110]:
training_data.take(5)

[LabeledPoint(0.0, [0.0,0.0,0.0,0.0,215.0,45076.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]),
 LabeledPoint(0.0, [0.0,0.0,0.0,0.0,162.0,4528.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0]),
 LabeledPoint(0.0, [0.0,0.0,0.0,0.0,236.0,1228.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,2.0,1.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0]),
 LabeledPoint(0.0, [0.0,0.0,0.0,0.0,233.0,2032.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,3.0,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0]),
 LabeledPoint(0.0, [0.0,0.0,0.0,0.0,239.0,486.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,4.0,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0])]

## Training a Classifier

In [111]:
# Build the model
# 3 categorical variable
t0 = time()
tree_model = DecisionTree.trainClassifier(training_data, numClasses=2, 
                                          categoricalFeaturesInfo={1: len(protocols), 2: len(services), 3: len(flags)},
                                          impurity='gini', maxDepth=4, maxBins=100)
tt = time() - t0

print("Classifier trained in {} seconds".format(round(tt,3)))

Classifier trained in 286.348 seconds


## Evaluating the Model

In [112]:
predictions = tree_model.predict(test_data.map(lambda p: p.features))
labels_and_preds = test_data.map(lambda p: p.label).zip(predictions)

In [113]:
# testing accuracy
t0 = time()
test_accuracy = labels_and_preds.filter(lambda x: x[0] == x[1]).count() / float(test_data.count())
tt = time() - t0

print("Prediction made in {} seconds. Test accuracy is {}".format(round(tt,3), round(test_accuracy,4)))

Prediction made in 24.898 seconds. Test accuracy is 0.9159


## Interpreting the Model

In [115]:
print("Learned classification tree model:")
print(tree_model.toDebugString())

Learned classification tree model:
DecisionTreeModel classifier of depth 4 with 25 nodes
  If (feature 22 <= 33.5)
   If (feature 38 <= 0.7949999999999999)
    If (feature 36 <= 0.49)
     If (feature 34 <= 0.9550000000000001)
      Predict: 0.0
     Else (feature 34 > 0.9550000000000001)
      Predict: 1.0
    Else (feature 36 > 0.49)
     If (feature 2 in {0.0,5.0,24.0,25.0,14.0,20.0,29.0,1.0,21.0,13.0,2.0,17.0,22.0,27.0,7.0,3.0,11.0,26.0,23.0,8.0,19.0,4.0})
      Predict: 0.0
     Else (feature 2 not in {0.0,5.0,24.0,25.0,14.0,20.0,29.0,1.0,21.0,13.0,2.0,17.0,22.0,27.0,7.0,3.0,11.0,26.0,23.0,8.0,19.0,4.0})
      Predict: 1.0
   Else (feature 38 > 0.7949999999999999)
    If (feature 3 in {0.0,1.0,6.0,2.0,8.0})
     Predict: 0.0
    Else (feature 3 not in {0.0,1.0,6.0,2.0,8.0})
     If (feature 36 <= 0.44)
      Predict: 1.0
     Else (feature 36 > 0.44)
      Predict: 1.0
  Else (feature 22 > 33.5)
   If (feature 5 <= 2.0)
    If (feature 2 in {11.0,66.0})
     Predict: 0.0
    Else 

In [23]:
print("Service 0 is {}".format(services[0]))
print("Service 52 is {}".format(services[52]))

Service 0 is http
Service 52 is pop_2


In [164]:
#tree_model.save(sc, "./myDecisionTreeClassificationModel")
#sameModel = DecisionTreeModel.load(sc, "./myDecisionTreeClassificationModel")

In [74]:
# reference 
# https://github.com/sanxofon/json_html_viewer/blob/master/json2js/json2jsondata.py
# https://github.com/tristaneljed/Decision-Tree-Visualization-Spark

# Parser
def parse(lines):
    block = []
    while lines:
        #import pdb; pdb.set_trace()
        if lines[0].startswith('If'):
            bl = ' '.join(lines.pop(0).split()[1:]).replace('(', '').replace(')', '')
            block.append({'name':bl, 'children':parse(lines)})
            if lines[0].startswith('Else'):
                be = ' '.join(lines.pop(0).split()[1:]).replace('(', '').replace(')', '')
                block.append({'name':be, 'children':parse(lines)})
        elif not lines[0].startswith(('If','Else')):
            block2 = lines.pop(0)
            block.append({'name':block2})
        else:
            break
    return block

# Convert Tree to JSON and save to structure.js file
def tree_json(tree):
    data = []
    for line in tree.splitlines() : 
        if line.strip():
            line = line.strip()
            data.append(line)
        else : break
        if not line : break
    res = []
    res.append({'name':'Root', 'children':parse(data[1:])}) # the recursive function parse each line of the result 
    #file = 'structure =' + str(res[0])
    with open('./structure.json', 'w') as outfile: # write tree as json structure to file
        json.dump(res[0], outfile)
    with open('./structure.json', 'r') as outfile: # open json file get data
        data=outfile.read()
    with open("./structure.js","w") as outfile: # rewrite as js file for visualization
        outfile.write("structure = '"+data +"';")

    print ('Conversion Success !')

In [108]:
## Create strucure.js file with the full-tree result
tree_json(tree_model.toDebugString())

Conversion Success !


## Building a Minimal Model Using the Three Main Splits

In [156]:
# create labelpoint object for minimal tree with 6 features

def create_labeled_point_minimal(line_split):
    # leave_out = [41]
    # only use feature [2,3,5,22,36,38]
    clean_line_split = line_split[2:4] + line_split[5:6] + line_split[22:23]+ line_split[36:37]+ line_split[38:39]
    
    # convert flag to numeric categorical variable
    try:
        clean_line_split[0] = services.index(clean_line_split[0])
    except:
        clean_line_split[0] = len(services)
        # convert flag to numeric categorical variable
    try:
        clean_line_split[1] = flags.index(clean_line_split[1])
    except:
        clean_line_split[1] = len(flags)
    
    # convert label to binary label
    attack = 1.0
    if line_split[41]=='normal.':
        attack = 0.0
        
    return LabeledPoint(attack, array([float(x) for x in clean_line_split]))

training_data_minimal = csv_data.map(create_labeled_point_minimal)
test_data_minimal = test_csv_data.map(create_labeled_point_minimal)

In [138]:
training_data_minimal.take(5)
len(services)

70

In [141]:
# Build the model
t0 = time()
tree_model_minimal = DecisionTree.trainClassifier(
    training_data_minimal, numClasses=2, 
    categoricalFeaturesInfo={0: len(services),1: len(flags)},
    impurity='gini', maxDepth=3, maxBins=70)
tt = time() - t0

print ("Classifier trained in {} seconds".format(round(tt,3)))

Classifier trained in 188.266 seconds


In [142]:
predictions_minimal = tree_model_minimal.predict(test_data_minimal.map(lambda p: p.features))
labels_and_preds_minimal = test_data_minimal.map(lambda p: p.label).zip(predictions_minimal)

### Check the testing accuracy
t0 = time()
test_accuracy = labels_and_preds_minimal.filter(lambda x: x[0] == x[1]).count() / float(test_data_minimal.count())
tt = time() - t0

print ("Prediction made in {} seconds. Test accuracy is {}".format(round(tt,3), round(test_accuracy,4)))

Prediction made in 15.666 seconds. Test accuracy is 0.9184


In [160]:
print("Learned classification minimal tree model:")
print(tree_model_minimal.toDebugString())
# feature 0: Services, 1: flag, 2: dst_bytes, 3: count, 4: dst_host_srv_diff_host_rate, 5: dst_host_srv_serror_rate

Learned classification minimal tree model:
DecisionTreeModel classifier of depth 3 with 15 nodes
  If (feature 3 <= 33.5)
   If (feature 5 <= 0.7949999999999999)
    If (feature 4 <= 0.49)
     Predict: 0.0
    Else (feature 4 > 0.49)
     Predict: 1.0
   Else (feature 5 > 0.7949999999999999)
    If (feature 1 in {0.0,1.0,6.0,2.0,8.0})
     Predict: 0.0
    Else (feature 1 not in {0.0,1.0,6.0,2.0,8.0})
     Predict: 1.0
  Else (feature 3 > 33.5)
   If (feature 2 <= 2.0)
    If (feature 0 in {11.0,66.0})
     Predict: 0.0
    Else (feature 0 not in {11.0,66.0})
     Predict: 1.0
   Else (feature 2 > 2.0)
    If (feature 0 in {0.0,2.0,8.0,10.0})
     Predict: 0.0
    Else (feature 0 not in {0.0,2.0,8.0,10.0})
     Predict: 0.0



### Export the minimal tree result to "structure.js" file

In [154]:
# create structure.js file with the result from minimal tree
tree_json(tree_model_minimal.toDebugString())

Conversion Success !


In [155]:
# Get label of freature to interpret the tree
print("Service 11 is {}".format(services[11]))
print("Service 66 is {}".format(services[66]))
print("Flags 0:2 is {}".format(flags[0:3]))
print("Flags 6 is {}".format(flags[6]))
print("Flags 8 is {}".format(flags[8]))
print("Flags 10 is {}".format(flags[10]))

Service 11 is urp_i
Service 66 is tftp_u
Flags 0:2 is ['SF', 'S2', 'S1']
Flags 6 is RSTO
Flags 8 is RSTR
Flags 10 is SH
